In [ ]:
# This program demonstrates a simple alhambra calculation by 
# creating a mix with a fixed concentration of MgCl2 and
# outputting the result using alhambra.
from alhambra_mixes import *

mg = Component("MgCl₂", "1 M")
h2o = Component("H₂0")

add_mg = FixedConcentration(mg, "125 mM")
mg_mix = Mix([add_mg], "10× Mg", fixed_total_volume="1 mL")

print(mg_mix)


In [ ]:
# This program creates a mix using alhambra and converts its output to
# a pandas DataFrame to display it using ipydatagrid.
from alhambra_mixes import Component, FixedConcentration, Mix
import pandas as pd
from ipydatagrid import DataGrid

# Create components
mg = Component("MgCl₂", "1 M")
h2o = Component("H₂0")

# Create a Mix
add_mg = FixedConcentration(mg, "125 mM")
mg_buffer = Mix([add_mg], "10× Mg", fixed_total_volume="1 mL")

# Convert the Mix output to a pandas DataFrame
mix_output = str(mg_buffer)
output_lines = mix_output.split("\n")[2:]
headers = output_lines[0].split("|")[1:-1]
headers = [header.strip() for header in headers]

data = []
for line in output_lines[2:]:
    row = line.split("|")[1:-1]
    row = [item.strip() for item in row]
    data.append(row)

df = pd.DataFrame(data, columns=headers)

# Display the DataFrame using ipydatagrid
datagrid = DataGrid(df)
datagrid




In [ ]:
# This program is a prototype for a more complex program that takes input data,
# creates alhambra Mixes based on the data, and displays the output in an ipydatagrid.
# The code is not currently functioning as expected and is only included here
# to show the steps taken in prototyping.
import re
import pandas as pd
from ipydatagrid import DataGrid, TextRenderer
from ipywidgets import VBox
from alhambra_mixes import Component as AlhambraComponent, FixedConcentration

class Component(AlhambraComponent):
    def __init__(self, name, concentration, location=None, well=None):
        super().__init__(name, concentration)
        self._location = location
        self._well = well

    @property
    def location(self):
        return self._location

    @location.setter
    def location(self, value):
        self._location = value

    @property
    def well(self):
        return self._well

    @well.setter
    def well(self, value):
        self._well = value

# Initialize the input DataFrame with a few empty rows
input_data = {
    'Name': ['MgCl₂', 'H₂0', '', '', ''],
    'Concentration': ['1 M', '', '', '', ''],
    'Location': ['', '', '', '', ''],
    'Well': ['', '', '', '', ''],
}
input_df = pd.DataFrame(input_data)

# Initialize the output DataFrame with column names
output_data = {
    'Component': [],
    'Concentration': [],
    'Location': [],
    'Well': [],
    'Mixing': [],
    'Volume': [],
    'Ratio': [],
    'Weight': [],
}
output_df = pd.DataFrame(output_data)

# Create DataGrids for input and output
input_datagrid = DataGrid(input_df, editable=True)
output_datagrid = DataGrid(output_df)

def mix_to_dict(mix):
    component_name = mix.components[0].name
    component_concentration = mix.components[0].concentration
    fixed_concentration = mix.fixed_concentration
    total_volume = mix.tx_volume
    number_of_components = len(mix.components)
    
    return {
        "Component": component_name,
        "Src": component_concentration,
        "Dest": fixed_concentration,
        "#": number_of_components,
        "Total Tx Vol": total_volume
    }


# Event handler for input data changes
def on_change(event=None):
    global output_df
    
    if event is not None:
        # Process the input DataFrame
        input_df = pd.DataFrame(event['data'])
    else:
        input_df = input_datagrid.data
    
    # Clear the output DataFrame
    #output_df = output_df.iloc[0:0]
    
    # Process the input DataFrame
    for idx, row in input_df.iterrows():
        # Create Components and Mixes based on the input data
        # Update the output DataFrame accordingly
        name = row['Name']
        concentration = row['Concentration']
        location = row['Location']
        well = row['Well']
        
        if not name or not concentration:
            continue
        
        component = Component(name, concentration, location, well)
        
        # Create Mixes based on the input data
        # For example, let's assume we want to make a FixedConcentration Mix for each component
        target_concentration = "125 mM"  # You can adjust this value based on your needs
        mix = FixedConcentration(component, target_concentration) 
        
        # Update the output DataFrame
        mix_dict = mix_to_dict(mix)
        mix_dict["Location"] = location
        mix_dict["Well"] = well

        print(mix_dict)
        
        output_df = pd.concat([output_df, pd.DataFrame([mix_dict])], ignore_index=True)

        print(output_df)
    
    # Update the output DataGrid
    output_datagrid.data = output_df

# Attach the on_change event handler to the input_datagrid
input_datagrid.observe(on_change, 'data')  # Use 'data' instead of input_df

# Call the on_change event handler manually to update the output spreadsheet
on_change()

# Display the input and output DataGrids in a VBox
VBox([input_datagrid, output_datagrid])


In [ ]:
# Incomplete prototype for parsing input from a spreadsheet and generating Alhambra mixes from it
import pandas as pd
from ipydatagrid import DataGrid
from ipywidgets import VBox
from alhambra_mixes import Component as AlhambraComponent, FixedConcentration

# class Component(AlhambraComponent):
#     def __init__(self, name, concentration, location=None, well=None):
#         super().__init__(name, concentration)
#         self._location = location
#         self._well = well

#     @property
#     def location(self):
#         return self._location

#     @location.setter
#     def location(self, value):
#         self._location = value

#     @property
#     def well(self):
#         return self._well

#     @well.setter
#     def well(self, value):
#         self._well = value

# Initialize the input DataFrame with a few empty rows
input_data = {
    'Name': ['MgCl₂', 'H₂0', '', '', ''],
    'Concentration': ['1 M', '', '', '', ''],
    'Location': ['', '', '', '', ''],
    'Well': ['', '', '', '', ''],
}
input_df = pd.DataFrame(input_data)

# Create DataGrid for input
input_datagrid = DataGrid(input_df, editable=True)

# Event handler for input data changes
def on_change(event=None):
    if event is not None:
        # Process the input DataFrame
        input_df = pd.DataFrame(event['data'])
    else:
        input_df = input_datagrid.data
    
    # df = pd.DataFrame(data, columns=headers)

    # Display the DataFrame using ipydatagrid
    datagrid = DataGrid(input_df)
    datagrid

    # Process the input DataFrame
    for idx, row in input_df.iterrows():
        name = row['Name']
        concentration = row['Concentration']
        location = row['Location']
        well = row['Well']
        
        if not name or not concentration:
            continue
        
        component = Component(name, concentration, location, well)
        
        # Create Mixes based on the input data
        # For example, let's assume we want to make a FixedConcentration Mix for each component
        target_concentration = "125 mM"  # You can adjust this value based on your needs
        mix = FixedConcentration(component, target_concentration)

# Attach the on_change event handler to the input_datagrid
input_datagrid.observe(on_change, 'data')  # Use 'data' instead of input_df

# Call the on_change event handler manually to update the output spreadsheet
on_change()

# Display the input DataGrid in a VBox
VBox([input_datagrid])




In [ ]:
# Single component calculation program with input form and output DataGrid, with some errors.
from alhambra_mixes import Component, FixedConcentration, Mix
import pandas as pd
from ipydatagrid import DataGrid
import ipywidgets as widgets

# Function to update the output DataGrid based on the input form
def update_output(*args):
    component_name = component_name_input.value
    component_conc = component_concentration_input.value
    fixed_conc = fixed_concentration_input.value
    total_vol = total_volume_input.value
    
    component = Component(component_name, component_conc)
    add_component = FixedConcentration(component, fixed_conc)
    mix = Mix([add_component], "Custom Mix", fixed_total_volume=total_vol)
    
    mix_output = str(mix)
    output_lines = mix_output.split("\n")[2:]
    headers = output_lines[0].split("|")[1:-1]
    headers = [header.strip() for header in headers]

    data = []
    for line in output_lines[2:]:
        row = line.split("|")[1:-1]
        row = [item.strip() for item in row]
        data.append(row)

    df = pd.DataFrame(data, columns=headers)
    output_datagrid.data = df

# Create input widgets
component_name_input = widgets.Text(description='Component Name:')
component_concentration_input = widgets.Text(description='Component Conc:')
fixed_concentration_input = widgets.Text(description='Fixed Conc:')
total_volume_input = widgets.Text(description='Total Volume:')

# Update output when input changes
component_name_input.observe(update_output, 'value')
component_concentration_input.observe(update_output, 'value')
fixed_concentration_input.observe(update_output, 'value')
total_volume_input.observe(update_output, 'value')

# Create input form
input_form = widgets.VBox([
    component_name_input,
    component_concentration_input,
    fixed_concentration_input,
    total_volume_input
])

# Create output DataGrid
output_datagrid = DataGrid(pd.DataFrame(columns=["Component", "[Src]", "[Dest]", "#", "Ea Tx Vol", "Tot Tx Vol", "Location", "Note"]))

# Display input form and output DataGrid
widgets.VBox([input_form, output_datagrid])


In [ ]:
# This program calculates a single component mix and displays the result in a DataGrid.
# It takes as input the component name, its concentration, target concentration for the mix,
# and the total volume of the mix. 
# Using the alhambra_mixes library, it creates a Component object, applies a FixedConcentration 
# action to it using the target concentration, and creates a Mix object using the action 
# and total volume. The output is then parsed into a pandas DataFrame and displayed in a DataGrid.
# If any input value is missing or invalid, an error message is displayed instead of the output.
# The program updates the output whenever the input values change using a reactive function.
from alhambra_mixes import Component, FixedConcentration, Mix
import pandas as pd
from ipydatagrid import DataGrid
import ipywidgets as widgets

def mix_to_dataframe(mix):
    mix_output = str(mix)
    output_lines = mix_output.split("\n")[2:]
    headers = output_lines[0].split("|")[1:-1]
    headers = [header.strip() for header in headers]

    data = []
    for line in output_lines[2:]:
        row = line.split("|")[1:-1]
        row = [item.strip() for item in row]
        data.append(row)

    df = pd.DataFrame(data, columns=headers)
    return df


def reactive_function(component_name, component_conc, fixed_conc, total_volume):
    try:
        # Check if any input value is empty or invalid
        if not all([component_name, component_conc, fixed_conc, total_volume]):
            raise ValueError("Some input values are missing or invalid")

        # Create components, action, and mix using alhambra
        component = Component(component_name, component_conc)
        action = FixedConcentration(component, fixed_conc)
        mix = Mix([action], "Example Mix", fixed_total_volume=total_volume)

        # Convert the mix output to a pandas DataFrame
        output_df = mix_to_dataframe(mix)

    except Exception as e:
        # If there's an error, display an appropriate message
        error_message = str(e)
        data = [["Error:", error_message]]
        output_df = pd.DataFrame(data, columns=["Error", "Message"])
    return output_df

def on_input_change(change):
    # Get current input values
    component_name = component_name_input.value
    component_conc = component_concentration_input.value
    fixed_conc = fixed_concentration_input.value
    total_volume = total_volume_input.value

    # Call the reactive function with the input values and update the output DataGrid
    output_df = reactive_function(component_name, component_conc, fixed_conc, total_volume)
    output_datagrid.data = output_df

# Create input widgets
component_name_input = widgets.Text(description='Component Name:')
component_concentration_input = widgets.Text(description='Component Conc:')
fixed_concentration_input = widgets.Text(description='Fixed Conc:')
total_volume_input = widgets.Text(description='Total Volume:')

# Update output when input changes
component_name_input.observe(on_input_change, 'value')
component_concentration_input.observe(on_input_change, 'value')
fixed_concentration_input.observe(on_input_change, 'value')
total_volume_input.observe(on_input_change, 'value')

# Create input form
input_form = widgets.VBox([
    component_name_input,
    component_concentration_input,
    fixed_concentration_input,
    total_volume_input
])

# Create output DataGrid
output_datagrid = DataGrid(pd.DataFrame(columns=["Component", "[Src]", "[Dest]", "#", "Ea Tx Vol", "Tot Tx Vol", "Location", "Note"]))

# Display input form and output DataGrid
widgets.VBox([input_form, output_datagrid])


In [ ]:
# This program shows an example of alhambra output with 2 components,
# which includes an error in the amount of buffer to add due to rounding.
from alhambra_mixes import *

# Components
mg = Component("MgCl₂", "1 M")
te = Component("100 X TE", "100 M")
h2o = Component("H₂0")

# Actions
add_mg = FixedConcentration(mg, "125 mM")
add_te = FixedConcentration(te, "1 M")

# Mix
mg_te_mix = Mix([add_mg, add_te], "Mg & TE Mix", fixed_total_volume="5 mL")

# Print the mix
print(mg_te_mix)

In [ ]:
# This program calculates a mix of components with fixed concentrations using
# the Alhambra library and displays the results in a data grid. Note that this
# program does not handle errors. Use the one below which has error handling.
from alhambra_mixes import Component, FixedConcentration, Mix
import pandas as pd
from ipydatagrid import DataGrid
import ipywidgets as widgets

def mix_to_dataframe(mix):
    mix_output = str(mix)
    output_lines = mix_output.split("\n")[2:]
    headers = output_lines[0].split("|")[1:-1]
    headers = [header.strip() for header in headers]

    data = []
    for line in output_lines[2:]:
        row = line.split("|")[1:-1]
        row = [item.strip() for item in row]
        data.append(row)

    df = pd.DataFrame(data, columns=headers)
    return df

def reactive_function(components_data, total_volume):
    try:
        # Check if any input value is empty or invalid
        if not all([total_volume]) or not all(components_data):
            raise ValueError("Some input values are missing or invalid")

        # Create components, actions, and mix using alhambra
        actions = []
        for data in components_data:
            component = Component(data['name'], data['component_conc'])
            action = FixedConcentration(component, data['fixed_conc'])
            actions.append(action)
        
        mix = Mix(actions, "Example Mix", fixed_total_volume=total_volume)

        # Convert the mix output to a pandas DataFrame
        output_df = mix_to_dataframe(mix)

    except Exception as e:
        # If there's an error, display an appropriate message
        error_message = str(e)
        data = [["Error:", error_message]]
        output_df = pd.DataFrame(data, columns=["Error", "Message"])

    # Format the 'Tot Tx Vol' column with 3 decimal places
    output_df['Tot Tx Vol'] = output_df['Tot Tx Vol'].apply(lambda x: '{:.3f}'.format(float(x)) if x.replace('.', '', 1).isdigit() else x)
    return output_df


def on_input_change(change):
    input_values = {
        'components': [
            {
                'name': component_name_input.value,
                'concentration': component_concentration_input.value,
                'fixed_concentration': fixed_concentration_input.value
            }
            for component_name_input, component_concentration_input, fixed_concentration_input
            in zip(component_name_inputs, component_concentration_inputs, fixed_concentration_inputs)
        ],
        'total_volume': total_volume_input.value
    }
    output_df = reactive_function(input_values)
    output_datagrid.data = output_df

# Set the number of components
num_components = 2

# Create input widgets
component_name_inputs = [widgets.Text(description='Component Name {}:'.format(i+1)) for i in range(num_components)]
component_concentration_inputs = [widgets.Text(description='Component Conc {}:'.format(i+1)) for i in range(num_components)]
fixed_concentration_inputs = [widgets.Text(description='Fixed Conc {}:'.format(i+1)) for i in range(num_components)]
total_volume_input = widgets.Text(description='Total Volume:')

# Update output when input changes
for component_name_input, component_concentration_input, fixed_concentration_input in zip(component_name_inputs, component_concentration_inputs, fixed_concentration_inputs):
    component_name_input.observe(on_input_change, 'value')
    component_concentration_input.observe(on_input_change, 'value')
    fixed_concentration_input.observe(on_input_change, 'value')
total_volume_input.observe(on_input_change, 'value')

# Create input form
input_form = widgets.VBox([
    *[
        widgets.HBox([
            component_name_input,
            component_concentration_input,
            fixed_concentration_input
        ])
        for component_name_input, component_concentration_input, fixed_concentration_input
        in zip(component_name_inputs, component_concentration_inputs, fixed_concentration_inputs)
    ],
    total_volume_input
])

# Create output DataGrid
output_datagrid = DataGrid(pd.DataFrame(columns=["Component", "[Src]", "[Dest]", "#", "Ea Tx Vol", "Tot Tx Vol", "Location", "Note"]))

# Display input form and output DataGrid
widgets.VBox([input_form, output_datagrid])


In [ ]:
# This program allows the user to calculate a mix of components with fixed concentrations
# using the Alhambra library, and displays the results in a data grid. The user can input 
# multiple components and their respective concentrations and fixed concentrations, 
# as well as the total volume of the mix. If any input value is invalid or missing, 
# the program displays an error message.
from alhambra_mixes import Component, FixedConcentration, Mix
import pandas as pd
from ipydatagrid import DataGrid
import ipywidgets as widgets


def mix_to_dataframe(mix):
    mix_output = str(mix)
    output_lines = mix_output.split("\n")[2:]
    headers = output_lines[0].split("|")[1:-1]
    headers = [header.strip() for header in headers]

    data = []
    for line in output_lines[2:]:
        row = line.split("|")[1:-1]
        row = [item.strip() for item in row]
        data.append(row)

    df = pd.DataFrame(data, columns=headers)
    return df

def reactive_function(input_values):
    try:
        # Check if any input value is empty or invalid
        if not all(input_values.values()):
            raise ValueError("Some input values are missing or invalid")

        # Create components, actions, and mix using alhambra
        actions = []
        for component_data in input_values['components']:
            component = Component(component_data['name'], component_data['concentration'])
            action = FixedConcentration(component, component_data['fixed_concentration'])
            actions.append(action)

        mix = Mix(actions, "Example Mix", fixed_total_volume=input_values['total_volume'])

        # Convert the mix output to a pandas DataFrame
        output_df = mix_to_dataframe(mix)

    except Exception as e:
        # If there's an error, display an appropriate message
        error_message = str(e)
        data = [["Error:", error_message]]
        output_df = pd.DataFrame(data, columns=["Error", "Message"])
    return output_df

def on_input_change(change):
    input_values = {
        'components': [
            {
                'name': component_name_input.value,
                'concentration': component_concentration_input.value,
                'fixed_concentration': fixed_concentration_input.value
            }
            for component_name_input, component_concentration_input, fixed_concentration_input
            in zip(component_name_inputs, component_concentration_inputs, fixed_concentration_inputs)
        ],
        'total_volume': total_volume_input.value
    }
    output_df = reactive_function(input_values)
    output_datagrid.data = output_df

# Set the number of components
num_components = 1

# Create input widgets
component_name_inputs = [widgets.Text(description='Component Name {}:'.format(i+1)) for i in range(num_components)]
component_concentration_inputs = [widgets.Text(description='Component Conc {}:'.format(i+1)) for i in range(num_components)]
fixed_concentration_inputs = [widgets.Text(description='Fixed Conc {}:'.format(i+1)) for i in range(num_components)]
total_volume_input = widgets.Text(description='Total Volume:')

# Update output when input changes
for component_name_input, component_concentration_input, fixed_concentration_input in zip(component_name_inputs, component_concentration_inputs, fixed_concentration_inputs):
    component_name_input.observe(on_input_change, 'value')
    component_concentration_input.observe(on_input_change, 'value')
    fixed_concentration_input.observe(on_input_change, 'value')
total_volume_input.observe(on_input_change, 'value')

# Create input form
input_form = widgets.VBox([
    *[
        widgets.HBox([
            component_name_input,
            component_concentration_input,
            fixed_concentration_input
        ])
        for component_name_input, component_concentration_input, fixed_concentration_input
        in zip(component_name_inputs, component_concentration_inputs, fixed_concentration_inputs)
    ],
    total_volume_input
])

# Create output DataGrid
output_datagrid = DataGrid(pd.DataFrame(columns=["Component", "[Src]", "[Dest]", "#", "Ea Tx Vol", "Tot Tx Vol", "Location", "Note"]))

# Display input form and output DataGrid
widgets.VBox([input_form, output_datagrid])


In [ ]:
# This program allows the user to calculate a mix of components with fixed concentrations
# using the Alhambra library, and displays the results in a data grid. The user can input 
# multiple components and their respective concentrations and fixed concentrations, 
# as well as the total volume of the mix. If any input value is invalid or missing, 
# the program displays an error message.
from alhambra_mixes import Component, FixedConcentration, Mix
import pandas as pd
from ipydatagrid import DataGrid
import ipywidgets as widgets


def mix_to_dataframe(mix):
    mix_output = str(mix)
    output_lines = mix_output.split("\n")[2:]
    headers = output_lines[0].split("|")[1:-1]
    headers = [header.strip() for header in headers]

    data = []
    for line in output_lines[2:]:
        row = line.split("|")[1:-1]
        row = [item.strip() for item in row]
        data.append(row)

    df = pd.DataFrame(data, columns=headers)
    return df

def reactive_function(samples_values):
    try:
        output_dfs = []
        for sample, input_values in samples_values.items():
            # Check if any input value is empty or invalid
            if not all(input_values.values()):
                raise ValueError("Some input values for sample {} are missing or invalid".format(sample))

            # Create components, actions, and mix using alhambra
            actions = []
            for component_data in input_values['components']:
                component = Component(component_data['name'], component_data['concentration'])
                action = FixedConcentration(component, component_data['fixed_concentration'])
                actions.append(action)

            mix = Mix(actions, "Sample {}".format(sample), fixed_total_volume=input_values['total_volume'])

            # Convert the mix output to a pandas DataFrame
            output_df = mix_to_dataframe(mix)
            output_df['Sample'] = sample
            output_dfs.append(output_df)

    except Exception as e:
        # If there's an error, display an appropriate message
        error_message = str(e)
        data = [["Error:", error_message]]
        output_df = pd.DataFrame(data, columns=["Error", "Message"])
        output_df['Sample'] = 'All'
        output_dfs.append(output_df)
    return pd.concat(output_dfs)

def on_input_change(change):
    sample = sample_selector.value
    samples_values[sample] = {
        'components': [
            {
                'name': component_name_input.value,
                'concentration': component_concentration_input.value,
                'fixed_concentration': fixed_concentration_input.value
            }
            for component_name_input, component_concentration_input, fixed_concentration_input
            in zip(component_name_inputs, component_concentration_inputs, fixed_concentration_inputs)
        ],
        'total_volume': total_volume_input.value
    }
    output_df = reactive_function(samples_values)
    output_datagrid.data = output_df

# Set the number of components
num_components = 1

# Create input widgets
component_name_inputs = [widgets.Text(description='Component Name {}:'.format(i+1)) for i in range(num_components)]
component_concentration_inputs = [widgets.Text(description='Component Conc {}:'.format(i+1)) for i in range(num_components)]
fixed_concentration_inputs = [widgets.Text(description='Fixed Conc {}:'.format(i+1)) for i in range(num_components)]
total_volume_input = widgets.Text(description='Total Volume:')

# Update output when input changes
for component_name_input, component_concentration_input, fixed_concentration_input in zip(component_name_inputs, component_concentration_inputs, fixed_concentration_inputs):
    component_name_input.observe(on_input_change, 'value')
    component_concentration_input.observe(on_input_change, 'value')
    fixed_concentration_input.observe(on_input_change, 'value')
total_volume_input.observe(on_input_change, 'value')

# Add a selector for the sample
sample_selector = widgets.Dropdown(options=['Sample 1', 'Sample 2', 'Sample 3', 'Sample 4'], description='Select Sample:')
for input_widget in component_name_inputs + component_concentration_inputs + fixed_concentration_inputs:
    input_widget.observe(on_input_change, 'value')

# Create input form
input_form = widgets.VBox([
    sample_selector,
    *[
        widgets.HBox([
            component_name_input,
            component_concentration_input,
            fixed_concentration_input
        ])
        for component_name_input, component_concentration_input, fixed_concentration_input
        in zip(component_name_inputs, component_concentration_inputs, fixed_concentration_inputs)
    ],
    total_volume_input
])

# Create output DataGrid
output_datagrid = DataGrid(pd.DataFrame(columns=["Component", "[Src]", "[Dest]", "#", "Ea Tx Vol", "Tot Tx Vol", "Location", "Note"]))


# Initialize a dictionary to hold sample values
samples_values = {sample: {} for sample in sample_selector.options}

# Display input form and output DataGrid
display(widgets.VBox([input_form, output_datagrid]))


In [ ]:
# This program allows the user to calculate a mix of components with fixed concentrations
# using the Alhambra library, and displays the results in a data grid. The user can input
# multiple components and their respective concentrations and fixed concentrations,
# as well as the total volume of the mix. If any input value is invalid or missing,
# the program displays an error message.
from alhambra_mixes import Component, FixedConcentration, Mix
import pandas as pd
from ipydatagrid import DataGrid
import ipywidgets as widgets

def mix_to_dataframe(mix):
    mix_output = str(mix)
    output_lines = mix_output.split("\n")[2:]
    headers = output_lines[0].split("|")[1:-1]
    headers = [header.strip() for header in headers]

    data = []
    for line in output_lines[2:]:
        row = line.split("|")[1:-1]
        row = [item.strip() for item in row]
        data.append(row)

    df = pd.DataFrame(data, columns=headers)
    return df

def reactive_function(samples_values):
    try:
        output_dfs = []
        for sample, input_values in samples_values.items():
            # Check if any input value is empty or invalid
            if not all(input_values.values()):
                raise ValueError("Some input values for sample {} are missing or invalid".format(sample))

            # Create components, actions, and mix using alhambra
            actions = []
            for component_data in input_values['components']:
                component = Component(component_data['name'], component_data['concentration'])
                action = FixedConcentration(component, component_data['fixed_concentration'])
                actions.append(action)

            mix = Mix(actions, "Sample {}".format(sample), fixed_total_volume=input_values['total_volume'])

            # Convert the mix output to a pandas DataFrame
            output_df = mix_to_dataframe(mix)
            output_df['Sample'] = sample
            output_dfs.append(output_df)

    except Exception as e:
        # If there's an error, display an appropriate message
        error_message = str(e)
        data = [["Error:", error_message]]
        output_df = pd.DataFrame(data, columns=["Error", "Message"])
        output_df['Sample'] = 'All'
        output_dfs.append(output_df)
    return pd.concat(output_dfs)

def on_input_change(change):
    sample = sample_selector.value
    samples_values[sample] = {
        'components': [
            {
                'name': component_name_input.value,
                'concentration': component_concentration_input.value,
                'fixed_concentration': fixed_concentration_input.value
            }
            for component_name_input, component_concentration_input, fixed_concentration_input
            in zip(component_name_inputs, component_concentration_inputs, fixed_concentration_inputs)
        ],
        'total_volume': total_volume_input.value
    }
    output_df = reactive_function(samples_values)
    output_datagrid.data = output_df

# Create input widgets
num_components_input = widgets.IntText(description='Number of Components:', value=2, style={'description_width': 'initial'})
total_volume_input = widgets.Text(description='Total Volume:', style={'description_width': 'initial'})

def on_num_components_change(change):
    num_components = change.new
    global component_name_inputs, component_concentration_inputs, fixed_concentration_inputs
    component_name_inputs = [widgets.Text(description=f'Component Name {i+1}:') for i in range(num_components)]
    component_concentration_inputs = [widgets.Text(description=f'Component Conc {i+1}:') for i in range(num_components)]
    fixed_concentration_inputs = [widgets.Text(description=f'Fixed Conc {i+1}:') for i in range(num_components)]

    # Update the input form
    input_form.children = [
        num_components_input,
        sample_selector,  # include sample_selector in the updated input_form
        *[
            widgets.HBox([
                component_name_input,
                component_concentration_input,
                fixed_concentration_input
            ])
            for component_name_input, component_concentration_input, fixed_concentration_input
            in zip(component_name_inputs, component_concentration_inputs, fixed_concentration_inputs)
        ],
        total_volume_input
    ]

    # Update output when input changes
    for component_name_input, component_concentration_input, fixed_concentration_input in zip(component_name_inputs, component_concentration_inputs, fixed_concentration_inputs):
        component_name_input.observe(on_input_change, 'value')
        component_concentration_input.observe(on_input_change, 'value')
        fixed_concentration_input.observe(on_input_change, 'value')
    total_volume_input.observe(on_input_change, 'value')


# Create input form with initial number of components
num_components = num_components_input.value
component_name_inputs = [widgets.Text(description=f'Component Name {i+1}:', style={'description_width': 'initial'}) for i in range(num_components)]
component_concentration_inputs = [widgets.Text(description=f'Component Conc {i+1}:', style={'description_width': 'initial'}) for i in range(num_components)]
fixed_concentration_inputs = [widgets.Text(description=f'Fixed Conc {i+1}:', style={'description_width': 'initial'}) for i in range(num_components)]

# Add a selector for the sample
sample_selector = widgets.Dropdown(options=['Sample 1', 'Sample 2', 'Sample 3', 'Sample 4'], description='Select Sample:')
for input_widget in component_name_inputs + component_concentration_inputs + fixed_concentration_inputs:
    input_widget.observe(on_input_change, 'value')

input_form = widgets.VBox([
    num_components_input,
    sample_selector,
    *[
        widgets.HBox([
            component_name_input,
            component_concentration_input,
            fixed_concentration_input
        ])
        for component_name_input, component_concentration_input, fixed_concentration_input
        in zip(component_name_inputs, component_concentration_inputs, fixed_concentration_inputs)
    ],
    total_volume_input
])


# Update output when input changes
for component_name_input, component_concentration_input, fixed_concentration_input in zip(component_name_inputs, component_concentration_inputs, fixed_concentration_inputs):
    component_name_input.observe(on_input_change, 'value')
    component_concentration_input.observe(on_input_change, 'value')
    fixed_concentration_input.observe(on_input_change, 'value')
total_volume_input.observe(on_input_change, 'value')

# Create output DataGrid
output_datagrid = DataGrid(pd.DataFrame(columns=["Component", "Tot Tx Vol"]))

# Attach on_num_components_change to num_components_input
num_components_input.observe(on_num_components_change, 'value')

# Initialize a dictionary to hold sample values
samples_values = {sample: {} for sample in sample_selector.options}

# Display input form and output DataGrid
widgets.VBox([input_form, output_datagrid])
